# Metagenomics

This Jupyternotebook is focusing on the analysis of metagenomics datasets. 
Metagenomes allow us to get a glimpse into not only composition but also functional potential of microbial communities.

We will use the PICRUST2 plugin to *infer metagenomic data* by mapping 16S rRNA gene sequences to their nearest matching whole genome sequences. For this, we will use the project datasets with the 16S rRNA genes. We will then use this data to gain insights into functional potential of our community. PICRUST2 will generate for us feature tables containing abundance information about KEGG orthologs, enzymes and entire pathways which we can later compare between samples and conditions.

**Notebook overview:**

[0. Setup](#setup)<br>
[1. Functional Inference](#picrust)<br>
&nbsp;&nbsp;&nbsp;&nbsp;[1.1 Enriched KEGG orthologs](#ipath)<br>
&nbsp;&nbsp;&nbsp;&nbsp;[1.2 Enriched pathways](#metacyc)<br>


Reference: PICRUST2 on its [GitHub wiki](https://github.com/picrust/picrust2/wiki), [this tutorial](https://github.com/picrust/picrust2/wiki/q2-picrust2-Tutorial) and the [Nature Biotechnology article](https://doi.org/10.1038/s41587-020-0548-6).

<a id='setup'></a>
## 0. Setup

In [2]:
import os
import pandas as pd
import qiime2 as q2
import requests
import math
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

from scipy import stats
from qiime2 import Visualization

data_dir = 'project_data'
    
%matplotlib inline

In [3]:
def fetch_ipath(ids: list, img_output_path: str, verbose: bool = False):
    """Fetches a enriched pathways map from iPATH3 for given IDs."""
    url = 'https://pathways.embl.de/mapping.cgi'
    
    # remove colon from EC names
    if ':' in ids[0]:
        ids = [x.replace(':', '') for x in ids]
    
    if verbose:
        print(f'Fetching iPATH3 diagram for ids: {ids}')
    params = {
        'default_opacity': 0.6,
        'export_type': 'svg',
        'selection': '\n'.join(ids)
    }   
    response = requests.get(url=url, params=params)
    
    with open(img_output_path, 'wb') as img:
        img.write(response.content)

In [4]:
# path to the picrust2 conda environment - do not change!
picrust_env = '/opt/conda/envs/picrust2/bin'

Download the `FeatureData[Sequence]` from our data which was made in the FirstLook.ipynb:

In [4]:
! wget -nv -O $data_dir/rep-seqs.qza 'https://polybox.ethz.ch/index.php/s/MBLSUQXzglnn66u/download?path=%2F&files=Sequences_rep_set.qza'

2022-11-28 15:50:29 URL:https://polybox.ethz.ch/index.php/s/MBLSUQXzglnn66u/download?path=%2F&files=Sequences_rep_set.qza [390624/390624] -> "project_data/rep-seqs.qza" [1]


Download the `FeatureTable[Frequency]` containing a mapping of the dereplicated sequences to samples from our data which was made in the FirstLook.ipynb:

In [5]:
! qiime tools peek $data_dir/rep-seqs.qza

UUID:        fd06ce7d-7b2d-4485-afda-fa50da61e9f4
Type:        FeatureData[Sequence]
Data format: DNASequencesDirectoryFormat


In [6]:
! wget -nv -O $data_dir/table.qza 'https://polybox.ethz.ch/index.php/s/MBLSUQXzglnn66u/download?path=%2F&files=Feature_table.qza'

2022-11-28 15:50:34 URL:https://polybox.ethz.ch/index.php/s/MBLSUQXzglnn66u/download?path=%2F&files=Feature_table.qza [504534/504534] -> "project_data/table.qza" [1]


Download the `Metadata` containing metadata to samples from our data:

In [7]:
! wget -nv -O $data_dir/metadata.tsv 'https://polybox.ethz.ch/index.php/s/QqbHeUIpIR0okB8/download'

2022-11-28 15:50:39 URL:https://polybox.ethz.ch/index.php/s/QqbHeUIpIR0okB8/download [300302/300302] -> "project_data/metadata.tsv" [1]


<a id='picrust'></a>
## 1. Functional inference

As mentioned in the introduction, we are using Picrust 2 to simulate metagenome data from our 16S dataset. Execute the command below to run picrust2 - it will take approximately 30-40 minutes.

In [13]:
%%script env picrust_env="$picrust_env" data_dir="$data_dir" bash

# append the env location to PATH so that qiime
# can find all required executables
export PATH=$picrust_env:$PATH


$picrust_env/qiime picrust2 full-pipeline \
    --i-seq $data_dir/rep-seqs.qza \
    --i-table $data_dir/table.qza \
    --output-dir $data_dir/picrust2_results \
    --p-placement-tool sepp \
    --p-threads 2 \
    --p-hsp-method pic \
    --p-max-nsti 2 --verbose

/opt/conda/envs/picrust2/bin:/opt/conda/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin
/opt/conda/envs/picrust2/bin
project_data
Saved FeatureTable[Frequency] to: project_data/picrust2_results/ko_metagenome.qza
Saved FeatureTable[Frequency] to: project_data/picrust2_results/ec_metagenome.qza
Saved FeatureTable[Frequency] to: project_data/picrust2_results/pathway_abundance.qza


Warning - 115 input sequences aligned poorly to reference sequences (--min_align option specified a minimum proportion of 0.8 aligning to reference sequences). These input sequences will not be placed and will be excluded from downstream steps.

This is the set of poorly aligned input sequences to be excluded: 13f72db00c0dbaa8958b9f7918f0b374, 07beb5cc8e914bcdb53a22bb47519c6f, 000fa3dd1e0addc3fe1e3b5d1008afe8, 94346c6d9a95d3b3ae8cc5199de67117, 879643dd982eb501f3c81fca990fd518, acb8def29a97a1c91b3fc3a21f2caf2b, b957bbc0b2cf69471ecf66e8ad3e04bc, aff251a2cb898bb774dbdb48905421cb, fbb21b0edc5b0ef81226200b39c869f6, 70e4caa9982037a944a5d98bf98d55f8, 9ce8bef85c4979f8b7bc3cac7ed448c3, b4eb3cde9bea75d4b44aa58ca81582c0, cd4a519b47d3968e0691867615a5c176, 50494e8139cc7865daf756431a9ddbc4, 6b3d79b96bbe4081267cbf253f6acf4a, 2d156bf28e5958f61156702359c1bbc9, d59db3377c485ff138fab36dfff0829a, cd0b6534edf1e39f840043cc9ff16245, b53340e67fe74981e67adddc676e0a70, aaf3da224d70d78e682e45aee3ffdd11, ea8eb307

In the `picrust2_results` subdirectory we get 3 artifacts - all of them are of the `FeatureTable[Frequency]` semantic type and hold abundance information for various functional features:

1. KO metagenome - abundances of [KEGG](https://www.kegg.jp) orthologs
2. EC metagenome - abundance of [enzymes](https://www.brenda-enzymes.org)
3. MetaCyc pathways - abundance of [MetaCyc](https://metacyc.org) pathways

In [23]:
metadata = pd.read_csv(f'{data_dir}/metadata.tsv', sep='\t', header=0, index_col=0)
metadata.index = metadata.index.astype(float)

In [7]:
print(metadata['NUT_alcohol_consumption'].value_counts())
print(metadata['NUT_alcohol_frequency'].value_counts())
print(metadata['NUT_diet_type'].value_counts())

True     415
False    108
Name: NUT_alcohol_consumption, dtype: int64
Occasionally    124
Regularly       122
Rarely          117
Never           103
Daily            52
Not provided      5
Name: NUT_alcohol_frequency, dtype: int64
Omnivore                            429
Omnivore but do not eat red meat     29
Vegetarian but eat seafood           27
Vegetarian                           24
Vegan                                 9
Not provided                          5
Name: NUT_diet_type, dtype: int64


Let's look at the metadata briefly to see what kind of categories we have available:

In [9]:
! qiime metadata tabulate \
    --m-input-file $data_dir/metadata.tsv \
    --o-visualization $data_dir/metadata.qzv

Saved Visualization to: project_data/metadata.qzv


In [10]:
Visualization.load(f'{data_dir}/metadata.qzv')

<visualization: Visualization uuid: da6f7794-fdf5-4c2a-b387-de64278373b4>

Now, we can read in all three artifacts using QIIME 2 Python API - we can view them as DataFrames:

In [24]:
ko = q2.Artifact.load(f'{data_dir}/picrust2_results/ko_metagenome.qza').view(pd.DataFrame)
ec = q2.Artifact.load(f'{data_dir}/picrust2_results/ec_metagenome.qza').view(pd.DataFrame)
pa = q2.Artifact.load(f'{data_dir}/picrust2_results/pathway_abundance.qza').view(pd.DataFrame)
ko.index = ko.index.astype(float)
ec.index = ec.index.astype(float)
pa.index = pa.index.astype(float)

<a id='ipath'></a>
### 1.1 Enriched KEGG orthologs visualization

Find the most abundant KEGG orthologs and plot them for both, where the sample alcohol cunsumption is .

We start by merging our feature table with the treatment column (`NUT_alcohol_consumption`) from the metadata:

In [25]:
ko_meta = ko.merge(metadata[['NUT_alcohol_consumption']], left_index=True, right_index=True)
ec_meta = ec.merge(metadata[['NUT_alcohol_consumption']], left_index=True, right_index=True)
pa_meta = pa.merge(metadata[['NUT_alcohol_consumption']], left_index=True, right_index=True)


Next, we will calculate an average abundance of each KO, EC and pathway in each group (alcohol consumption vs. no alcohol consumption):

In [27]:
# collapse samples per sample_type - calculate average abundance

ko_meta_avg = ko_meta.groupby('NUT_alcohol_consumption').mean()
ec_meta_avg = ec_meta.groupby('NUT_alcohol_consumption').mean()
pa_meta_avg = pa_meta.groupby('NUT_alcohol_consumption').mean()

In [28]:
ko_meta_avg.head()

,K00001,K00002,K00003,K00004,K00005,K00007,K00008,K00009,K00010,K00011,...,K19776,K19777,K19778,K19779,K19780,K19784,K19785,K19788,K19789,K19791
NUT_alcohol_consumption,,,,,,,,,,,,,,,,,,,,,
False,3162.421122,68.691026,4796.981177,398.456162,3772.020015,83.825277,4324.14981,1467.899032,2903.594500,0.339969,...,1083.124950,718.306041,573.952036,94.495166,137.206914,1247.168509,3.566936e-127,0.000026,1006.961259,0.006508
True,3181.198018,73.266389,4909.399083,400.607382,3893.175186,51.285504,4566.19005,1400.650732,2557.094055,0.792130,...,1075.325431,688.104082,612.494259,103.840925,109.865976,1059.402351,7.638711e-06,0.002091,922.806781,0.005347


Finally, let's find the most abundant features in each table:

In [29]:
# find top x% of the most abundant KOs, ECs and pathways in each sample type

def find_most_abundant(df: pd.DataFrame, frac):
    if 0 < frac < 1:
        frac = int(frac * len(df.columns))
    print(f'Saving {frac} most abundant features...')
    most_abundant = {
        smp: df.loc[smp, :].sort_values(ascending=False)[:frac]
        for smp in df.index
    }
    return most_abundant

ko_most_abundant = find_most_abundant(ko_meta_avg, 0.01)
ec_most_abundant = find_most_abundant(ec_meta_avg, 0.03)
pa_most_abundant = find_most_abundant(pa_meta_avg, 5)

Saving 101 most abundant features...
Saving 85 most abundant features...
Saving 5 most abundant features...


In [34]:
print(f'10 most abundant KOs in the alcohole consumption group are: {ko_most_abundant[True].index[:10].tolist()}\n'
      f'10 most abundant KOs in the non-alcohole consumption group are: {ko_most_abundant[False].index[:10].tolist()}\n')

10 most abundant KOs in the alcohole consumption group are: ['K03088', 'K01990', 'K02004', 'K06147', 'K01992', 'K02003', 'K02529', 'K07024', 'K03497', 'K02015']
10 most abundant KOs in the non-alcohole consumption group are: ['K03088', 'K01990', 'K02004', 'K01992', 'K06147', 'K02003', 'K02529', 'K07024', 'K00059', 'K02015']



To visualize the KOs and ECs we can use the [Interactive Pathway Explorer](https://pathways.embl.de) (iPath 3). First, we will fetch pathway maps for KOs and ECs per sample group and display them as SVGs. Then, you can try copying some IDs and trying out the interactive pathway map available on the web page linked above - it will allow you to zoom into different areas of the map and look at some more interesting details.

In [35]:
for smp in ko_most_abundant.keys():
    fetch_ipath(ko_most_abundant[smp].index.tolist(), f'{data_dir}/picrust2_results/kos_{smp}.svg')
    fetch_ipath(ec_most_abundant[smp].index.str.replace(':', '').tolist(), f'{data_dir}/picrust2_results/ecs_{smp}.svg')

This should have fetched 4 pathway maps: 1 map per sample group (alcohol cons vs. non-alcohol cons) and 1 map per feature table (KO vs. EC). The are rather large images and so displaying them inline here would make the maps unreadable.

<a id='metacyc'></a>
### 1.2 Enriched pathways

To get a bigger picture, we can also look at the most abundant pathways in both sample groups. Use one of the functions defined above to identify the four pathways that are most abundant in the alcohole and non-alcohole consumption.

In [40]:
print(f'4 most abundant pathways in the alcohol consumption group are: {pa_most_abundant[True].index[:4].tolist()}\n'
      f'4 most abundant pathways in the non-alcohol consumptiopn are: {pa_most_abundant[False].index[:4].tolist()}\n')

4 most abundant pathways in the treatment group are: ['NONOXIPENT-PWY', 'PWY-7111', 'PWY-5101', 'PWY-7663']
4 most abundant pathways in the non-treatment group are: ['NONOXIPENT-PWY', 'PWY-7111', 'PWY-5101', 'PWY-7663']



Find whether there are pathways that differ significantly between samples with and without alcohol consumption.

In [43]:
! qiime composition add-pseudocount \
    --i-table $data_dir/picrust2_results/pathway_abundance.qza \
    --o-composition-table $data_dir/picrust2_results/pathway_abundance_abund.qza

Saved FeatureTable[Composition] to: project_data/picrust2_results/pathway_abundance_abund.qza


In [44]:
! qiime composition ancom \
    --i-table $data_dir/picrust2_results/pathway_abundance_abund.qza \
    --m-metadata-file $data_dir/metadata.tsv \
    --m-metadata-column 'NUT_alcohol_consumption' \
    --p-transform-function log \
    --o-visualization $data_dir/pa_ancom_alc.qzv

Saved Visualization to: project_data/pa_ancom_alc.qzv


In [45]:
Visualization.load(f'{data_dir}/pa_ancom_alc.qzv')

<visualization: Visualization uuid: e6b26a88-53ec-419b-93ed-e70073030e5b>